# Lang graph Quickstart Guide

In [1]:
from langgraph.prebuilt import create_react_agent
import os
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel
from langchain.tools import tool

In [3]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq",
    temperature=0
)

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt="You are a helpful assistant. Use the tool `get_weather` to answer weather-related questions. End the conversation once the user’s question is answered.",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='afdafe43-b1b1-4ac4-b5fb-f7a2bd66c59f'),
  AIMessage(content='<function=get_weather>{"city": "sf"}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 248, 'total_tokens': 259, 'completion_time': 0.014666667, 'prompt_time': 0.011652067, 'queue_time': 0.20969699900000002, 'total_time': 0.026318734}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f7bd09b454', 'finish_reason': 'stop', 'logprobs': None}, id='run--5b82a230-90a1-482b-a451-5b32aacef8f6-0', usage_metadata={'input_tokens': 248, 'output_tokens': 11, 'total_tokens': 259})]}

Static Prompting

In [5]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq",
    temperature=0
)

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    # A static prompt that never changes
    prompt="Never answer questions about the weather."
)
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)


{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='2e2fb6d7-b0d3-4ac9-826f-19ec04c557af'),
  AIMessage(content='<function=get_weather>{"city": "sf"}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 225, 'total_tokens': 236, 'completion_time': 0.014666667, 'prompt_time': 0.007364503, 'queue_time': 0.212876574, 'total_time': 0.02203117}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f7bd09b454', 'finish_reason': 'stop', 'logprobs': None}, id='run--5425f429-bfc3-47ef-86e9-0ddabc9ac9e1-0', usage_metadata={'input_tokens': 225, 'output_tokens': 11, 'total_tokens': 236})]}

Dynamic prompting 

In [7]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    user_name = config["configurable"].get("user_name")
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq",
    temperature=0
)

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    # A static prompt that never changes
    prompt=prompt
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config={"configurable": {"user_name": "John Smith"}}
)


{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='9b5d90a0-e0a3-44d3-89f4-cf5cd704e41e'),
  AIMessage(content='<function=get_weather>{"city": "sf"}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 231, 'total_tokens': 242, 'completion_time': 0.014666667, 'prompt_time': 0.017609586, 'queue_time': 0.467000629, 'total_time': 0.032276253}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'stop', 'logprobs': None}, id='run--e7af5d31-41bb-4de7-925b-491d8f72f472-0', usage_metadata={'input_tokens': 231, 'output_tokens': 11, 'total_tokens': 242})]}

Memory 

In [8]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq",
    temperature=0
)

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

Configure Structured Output 

In [ ]:
from pydantic import BaseModel

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq",
    temperature=0
)

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    response_format=WeatherResponse
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)